In [10]:
import pytesseract
from pdf2image import convert_from_path

In [11]:
pdf_path = "../../assets/pdfs/SamplePDFWithWords.pdf"
page_images = convert_from_path(pdf_path)

In [ ]:
# NOTE: This sample seems to miss some of the numbers on the first page.
for page_image in page_images:
	text = pytesseract.image_to_string(page_image)
	print("PAGE:")
	print(text)

PAGE:
Here is a sample document that has words in it. And, here are some more words. And some
more.

1

2

20
21
22
23

24

PAGE:
25
26

27

Now, there will be a table.

Food Primary Color Category
Apple Red Fruit
Carrot Orange Vegetable


